In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
import os

# Load the dataset
file_path = "../../data/clean_data/AIDSVu_for_links.csv"
data = pd.read_csv(file_path)

# Function to create a color scale from light pink to dark red
def create_color_scale(value, min_val, max_val, n_colors=10):
    # Create color map from light pink to dark red
    # These colors should match the CSS classes in the HTML
    # Made the lightest pink darker for better visibility
    colors = [
        '#f8d7da', '#ffb6c1', '#ef9a9a', '#e57373',
        '#ef5350', '#f44336', '#e53935', '#d32f2f',
        '#c62828', '#b71c1c'
    ]

    # Calculate which color to use based on the value
    if max_val == min_val:
        return colors[0]

    # Normalize based on custom min_val and max_val
    normalized = (value - min_val) / (max_val - min_val)
    # Clamp the value between 0 and 1
    normalized = max(0, min(normalized, 1))
    color_index = min(int(normalized * len(colors)), len(colors) - 1)
    return colors[color_index]

# =====================
# TABLE 1: New Diagnoses by Race
# =====================

# Filter for New Diagnoses, Total Age, Total Sex
new_diagnoses = data[(data['Indicator'] == 'New Diagnoses') &
                     (data['Age'] == 'Total') &
                     (data['Sex'] == 'Total')]

# Define the preferred row order (Total first, then by descending Rate)
preferred_order = ['Total']
# Get all other races sorted by Rate (descending)
other_races = new_diagnoses[new_diagnoses['Race'] != 'Total'][['Race', 'Rate']].sort_values('Rate', ascending=False)['Race'].tolist()
# Complete the race order
race_order = preferred_order + other_races

# Sort the data according to the preferred order
new_diagnoses_table = pd.DataFrame()
for race in race_order:
    row = new_diagnoses[new_diagnoses['Race'] == race]
    if not row.empty:
        new_diagnoses_table = pd.concat([new_diagnoses_table, row])

# Set custom min and max values for color scaling - CUSTOM RANGE as requested
min_rate = 0  # Custom min for Table 1
max_rate = 70  # Custom max for Table 1

# Create HTML for Table 1
table1_html = """
<h2>Table 1: New HIV Diagnoses Rate by Race</h2>
<table border="1" style="border-collapse: collapse; width: 100%;">
  <thead>
    <tr style="background-color: #4a5568; color: #fff;">
      <th style="padding: 12px 15px; text-align: left;">Race</th>
      <th style="padding: 12px 15px; text-align: left;">Rate (per 100,000)</th>
    </tr>
  </thead>
  <tbody>
"""

# Add rows with background color based on rate
previous_is_total = False
for _, row in new_diagnoses_table.iterrows():
    bg_color = create_color_scale(row['Rate'], min_rate, max_rate)

    # Add thicker border after Total
    if previous_is_total:
        top_border = 'border-top: 3px solid #666;'
    else:
        top_border = ''

    # Bold text for Total
    if row['Race'] == 'Total':
        race_style = 'font-weight: bold; color: #000;'
        rate_style = 'font-weight: bold; color: #000;'
        previous_is_total = True
    else:
        race_style = 'color: #000;'
        rate_style = 'color: #000;'

    table1_html += f"""
    <tr style="background-color: {bg_color};">
      <td style="padding: 12px 15px; border-bottom: 1px solid #ddd; {top_border} {race_style}">{row['Race']}</td>
      <td style="padding: 12px 15px; border-bottom: 1px solid #ddd; {top_border} {rate_style}">{row['Rate']}</td>
    </tr>
    """

table1_html += """
  </tbody>
</table>
"""

# =====================
# TABLE 2: MSM by Race
# =====================

# Filter for MSM, Total Age, Total Sex
msm_data = data[(data['Indicator'] == 'MSM') &
               (data['Age'] == 'Total') &
               (data['Sex'] == 'Total')]

# Use the same race order as in Table 1
msm_table = pd.DataFrame()
for race in race_order:
    row = msm_data[msm_data['Race'] == race]
    if not row.empty:
        msm_table = pd.concat([msm_table, row])

# Set custom min and max values for color scaling - CUSTOM RANGE
min_msm = 50  # Custom min for Table 2
max_msm = 80  # Custom max for Table 2

# Create HTML for Table 2 (MSM)
table2_html = """
<h3>Table 2: Men who have Sex with Men (MSM)</h3>
<table border="1" style="border-collapse: collapse; width: 100%;">
  <thead>
    <tr style="background-color: #4a5568; color: #fff;">
      <th style="padding: 12px 15px; text-align: left;">Race</th>
      <th style="padding: 12px 15px; text-align: left;">MSM (%)</th>
    </tr>
  </thead>
  <tbody>
"""

# Add rows with background color based on MSM percentage
previous_is_total = False
for _, row in msm_table.iterrows():
    # Use the custom min_msm and max_msm values here:
    bg_color = create_color_scale(row['Percent'], min_msm, max_msm)
    
    # Add thicker border after Total
    if previous_is_total:
        top_border = 'border-top: 3px solid #666;'
    else:
        top_border = ''

    # Bold text for Total
    if row['Race'] == 'Total':
        race_style = 'font-weight: bold; color: #000;'
        percent_style = 'font-weight: bold; color: #000;'
        previous_is_total = True
    else:
        race_style = 'color: #000;'
        percent_style = 'color: #000;'

    table2_html += f"""
    <tr style="background-color: {bg_color};">
      <td style="padding: 12px 15px; border-bottom: 1px solid #ddd; {top_border} {race_style}">{row['Race']}</td>
      <td style="padding: 12px 15px; border-bottom: 1px solid #ddd; {top_border} {percent_style}">{row['Percent']}</td>
    </tr>
    """

table2_html += """
  </tbody>
</table>
"""

# =====================
# TABLE 3: IDU by Race
# =====================

# Filter for IDU, Total Age, Total Sex
idu_data = data[(data['Indicator'] == 'IDU') &
               (data['Age'] == 'Total') &
               (data['Sex'] == 'Total')]

# Use the same race order as in Table 1
idu_table = pd.DataFrame()
for race in race_order:
    row = idu_data[idu_data['Race'] == race]
    if not row.empty:
        idu_table = pd.concat([idu_table, row])

# Set custom min and max values for color scaling - CUSTOM RANGE
min_idu = 0  # Custom min for Table 3
max_idu = 30  # Custom max for Table 3

# Create HTML for Table 3 (IDU)
table3_html = """
<h3>Table 3: Injection Drug Use (IDU)</h3>
<table border="1" style="border-collapse: collapse; width: 100%;">
  <thead>
    <tr style="background-color: #4a5568; color: #fff;">
      <th style="padding: 12px 15px; text-align: left;">Race</th>
      <th style="padding: 12px 15px; text-align: left;">IDU (%)</th>
    </tr>
  </thead>
  <tbody>
"""

# Add rows with background color based on IDU percentage
previous_is_total = False
for _, row in idu_table.iterrows():
    # Use the custom min_idu and max_idu values here:
    bg_color = create_color_scale(row['Percent'], min_idu, max_idu)
    
    # Add thicker border after Total
    if previous_is_total:
        top_border = 'border-top: 3px solid #666;'
    else:
        top_border = ''

    # Bold text for Total
    if row['Race'] == 'Total':
        race_style = 'font-weight: bold; color: #000;'
        percent_style = 'font-weight: bold; color: #000;'
        previous_is_total = True
    else:
        race_style = 'color: #000;'
        percent_style = 'color: #000;'

    table3_html += f"""
    <tr style="background-color: {bg_color};">
      <td style="padding: 12px 15px; border-bottom: 1px solid #ddd; {top_border} {race_style}">{row['Race']}</td>
      <td style="padding: 12px 15px; border-bottom: 1px solid #ddd; {top_border} {percent_style}">{row['Percent']}</td>
    </tr>
    """

table3_html += """
  </tbody>
</table>
"""

# Combine tables for Table 1 HTML file
table1_full_html = f"""
<!DOCTYPE html>
<html>
<head>
  <style>
    body {{
      font-family: Arial, sans-serif;
      background-color: #121212;
      color: #fff;
      max-width: 1200px;
      margin: 0 auto;
      padding: 20px;
    }}
    h1, h2, h3 {{
      color: #fff;
    }}
    h3 {{
      font-size: 1.1em;
      margin-top: 10px;
    }}
    table {{
      width: 100%;
      box-shadow: 0 4px 8px rgba(0,0,0,0.1);
      margin-bottom: 30px;
    }}
    th, td {{
      padding: 12px 15px;
      text-align: left;
      border-bottom: 1px solid #ddd;
      color: #000;
    }}
    th {{
      background-color: #4a5568;
      color: #fff;
      font-weight: bold;
    }}
    tr:hover {{
      filter: brightness(85%);
    }}
  </style>
</head>
<body>
  {table1_html}
</body>
</html>
"""

# Combine tables for Tables 2 & 3 HTML file
tables23_full_html = f"""
<!DOCTYPE html>
<html>
<head>
  <style>
    body {{
      font-family: Arial, sans-serif;
      background-color: #121212;
      color: #fff;
      max-width: 1200px;
      margin: 0 auto;
      padding: 20px;
    }}
    h1, h2, h3 {{
      color: #fff;
    }}
    h3 {{
      font-size: 1.1em;
      margin-top: 10px;
    }}
    table {{
      width: 100%;
      box-shadow: 0 4px 8px rgba(0,0,0,0.1);
      margin-bottom: 30px;
    }}
    th, td {{
      padding: 12px 15px;
      text-align: left;
      border-bottom: 1px solid #ddd;
      color: #000;
    }}
    th {{
      background-color: #4a5568;
      color: #fff;
      font-weight: bold;
    }}
    tr:hover {{
      filter: brightness(85%);
    }}
    .tables-container {{
      display: flex;
      justify-content: space-between;
      flex-wrap: wrap;
    }}
    .table-wrapper {{
      width: 48%;
    }}
    @media (max-width: 768px) {{
      .table-wrapper {{
        width: 100%;
      }}
    }}
  </style>
</head>
<body>
  <h2>Transmission Mode by Race (%)</h2>
  <div class="tables-container">
    <div class="table-wrapper">
      {table2_html}
    </div>
    <div class="table-wrapper">
      {table3_html}
    </div>
  </div>
</body>
</html>
"""

# Save the HTML files to separate files
# Create the directory if it doesn't exist
os.makedirs('../../interactive_viz_outputs', exist_ok=True)

with open('../../interactive_viz_outputs/table_rates.html', 'w') as f:
    f.write(table1_full_html)

with open('../../interactive_viz_outputs/tables_transmission.html', 'w') as f:
    f.write(tables23_full_html)

print("HTML files created successfully in ../../interactive_viz_outputs/")

HTML files created successfully in ../../interactive_viz_outputs/
